In [29]:
import fitbit

In [44]:
api = fitbit.API(debug=True)

Visit the following url to get your auth code: https://www.fitbit.com/oauth2/authorize?response_type=code&client_id=23BB84&redirect_uri=http%3A%2F%2Flocalhost&scope=activity+nutrition+heartrate+location+nutrition+profile+settings+sleep+social+weight
Auth code: 581d62569dee81369ed10b4a17913002310011fe


In [63]:
res = api.activity_time_series("calories", "2021-09-01", "7d")
res.json()

{'activities-calories': [{'dateTime': '2021-08-26', 'value': '2879'},
  {'dateTime': '2021-08-27', 'value': '2267'},
  {'dateTime': '2021-08-28', 'value': '4038'},
  {'dateTime': '2021-08-29', 'value': '3249'},
  {'dateTime': '2021-08-30', 'value': '2389'},
  {'dateTime': '2021-08-31', 'value': '2950'},
  {'dateTime': '2021-09-01', 'value': '2277'}]}

{'goals': {'distance': 56.33, 'floors': 70, 'steps': 70000}}